# Aligning NIRSpec MOS Spectra using Spectral Features

In this notebook, we will demonstrate how to align two NIRSpec MOS spectra using spectral features. An example might be for two observations of the same galaxy observed through different MSA shutters. This process can be used to check WCS solutions, or to fine-tune the spectral registration.

## Table of Contents:
> * [Imports](#imports)
> * [Reading and Selecting the spectra](#Reading)
> * [Extracting 1D Spectra](#Extracting)
> * [Isolating regions of overlap](#Isolating)
> * [Normalizing the continuum](#Normalizing)
> * [Registering the second spectrum to the first in wavelength](#Registering)
> * [Trying again](#Trying_again)
> * [Version information](#Versions)
> * [About this notebook](#About)


***
<a id='imports'></a>
## Imports

* `astropy.io fits` for accessing FITS files
* `astropy.modeling` for fitting to get a new dispersion solution for the spectrum being registered
* `numpy` numerical library used for array manipulation
* `matplotlib.pyplot` for plotting data
* `scipy.interpolate` used to interpolate one spectrum on to the wavelength grid of the other
* `scipy.optimize` for fitting the dispersion solution
* `matplotlib inline` to display plots inline

In [ ]:
from astropy.io import fits
from astropy.modeling import models, fitting
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import minimize
%matplotlib inline

***
<a id='Reading'></a>
## Reading and selecting the spectra

For this demonstration, we'll use some sample data which has been run partway through the pipeline, ending with extract2d. This means that the spectra have not been rectified. If you use data which has already been extracted as 1D spectra, you can skip the next few steps (but then, the spectra might already line up for you, so this wouldn't be demonstrating how to do it).

Let's open the file and inspect to decide which extension to use. After a bit of iteration through the next few cells, we decide on extensions 9 and 23, which have enough pixels to work with.

In [ ]:
# Original filename = 'F170LP-G235M_MOS_observation-6-c0e0_001_DN_NRS1_mod_updatedHDR_cal.fits'
filename='https://stsci.box.com/shared/static/fvrgpesqrpbzdv5btn17pb899btwd61a.fits'
fits.info(filename)  # What extensions should we use?

In [ ]:
# These are two science extensions with sufficient pixels
ext1 = 9
ext2 = 23

sci1 = fits.getdata(filename, ext1)
sci2 = fits.getdata(filename, ext2)

fig1, ax1 = plt.subplots(figsize=(10, 2))
tmp = ax1.imshow(sci1, origin='lower', interpolation='none')
fig2, ax2 = plt.subplots(figsize=(10, 2))
tmp = ax2.imshow(sci2, origin='lower', interpolation='none')

***
<a id='Extracting'></a>
## Extracting 1D spectra

We'll pretend that these are already rectified, and do a simple sum over the cross-dispersion 
direction to work with 1d spectra.

In [ ]:
spec1 = sci1.sum(axis=0)
spec2 = sci2.sum(axis=0)
fig3 = plt.figure(figsize=(8, 4))
tmp = plt.plot(spec1)
tmp = plt.plot(spec2)
plt.xlabel('pixels')

The wavelength extensions are also 2D, so we'll use a median (ignoring NaN values) to get 1-D wavelength arrays.

In [ ]:
wav1 = fits.getdata(filename, ext1+3)
wav2 = fits.getdata(filename, ext2+3)

wave1 = np.nanmedian(wav1, axis=0)
wave2 = np.nanmedian(wav2, axis=0)

fig4 = plt.figure(figsize=(8, 4))
tmp = plt.plot(wave1, spec1)
tmp = plt.plot(wave2, spec2)
plt.xlabel('microns')

***
<a id='Isolating'></a>
## Isolate regions of overlap

Since the spectra are different shapes, we need to isolate the portions of the spectra where they overlap.

In [ ]:
wave_lo, wave_hi = max(wave1.min(), wave2.min()), min(wave1.max(), wave2.max())
in_wave1 = (wave1 >= wave_lo) & (wave1 <= wave_hi) & (spec1 != 0.0)
in_wave2 = (wave2 >= wave_lo) & (wave2 <= wave_hi) & (spec2 != 0.0)

fig5 = plt.figure(figsize=(8, 4))
tmp = plt.plot(wave1[in_wave1], spec1[in_wave1])
tmp = plt.plot(wave2[in_wave2], spec2[in_wave2])

***
<a id='Normalizeing'></a>
## Normalize the continuum

We also need to normalize the fluxes to make our wavelength fits better. First, inspect the ratio of the second spectrum to the first. To do this, we need to interpolate our second spectrum onto the same wavelength grid, which we will do using simple linear interpolation.

In [ ]:
out_wave = wave1[in_wave1]
interp_spec2 = interp1d(wave2[in_wave2], spec2[in_wave2], fill_value='extrapolate')(out_wave)
div_spec = interp_spec2 / spec1[in_wave1]

fig6 = plt.figure(figsize=(8, 4))
tmp = plt.plot(out_wave, div_spec)

It looks like a 2nd-order polynomial, starting redwards of 1.70 microns might do a good job of fitting this ratio (ignoring the lines).  If the spectra are not already well-aligned, more robust fitting methods, probably including outlier rejection, would be necessary. We'll do the fit using the astropy `Polynomial1D` model and a linear least-squares fitter. 

In [ ]:
norm_model = models.Polynomial1D(2)
fitter = fitting.LinearLSQFitter()

ok_px = (np.isfinite(div_spec)) & (out_wave >= 1.70)

normalization = fitter(norm_model, out_wave[ok_px], div_spec[ok_px])
print(normalization)

flux_scale_factor = normalization(out_wave)
scaled_spec1 = spec1[in_wave1] * flux_scale_factor

# Plot the fit
fig7 = plt.figure(figsize=(8,4))
tmp = plt.plot(out_wave, div_spec)
tmp = plt.plot(out_wave, flux_scale_factor, '--')


# Plot the normalized spectra 
fig8 = plt.figure(figsize=(8, 4))
tmp = plt.plot(out_wave, scaled_spec1)
tmp = plt.plot(out_wave, interp_spec2)

***
<a id='Registering'></a>
## Register the second spectrum to the first in wavelength

To register the second spectrum to the first in wavelength, we basically need to warp its dispersion solution. 
We do this using `scipy.optimize.minimize`, where 
we'll transform one of the wavelength arrays with a polynomial, then use that new wavelength
to resample the associated spectrum to the reference wavelengths. Our metric is the standard
deviation of the difference spectrum, which we want to minimize.

First, create a few helper functions for plotting purposes.

In [ ]:
# Give the old wavelength and the coefficients of the fit, return the new wavelength
def alter_wave(poly_coefs, old_wave): 
    altered_wave2 = np.zeros_like(old_wave)
    for c, coef in enumerate(poly_coefs):  # this can be something other than a polynomial, if desired
        altered_wave2 += coef * old_wave**c
    return altered_wave2

# Interpolate the second spectrum onto the wavelength grid of the first
def alter_spec(wave): 
    altered_spec2 = interp1d(wave, spec2[in_wave2], fill_value='extrapolate')(out_wave)
    return altered_spec2

Create a function (to be passed to scipy's `minimize` routine) to compute the quantity that we wish to optimize (by warping the dispersion solution of the second spectrum). We choose to minimize the standard deviation of the differences between the two spectra.

In [ ]:
def fit_metric(poly_coefs, old_wave):
    altered_wave2 = alter_wave(poly_coefs, old_wave)  # construct the new wavelength array
    altered_spec2 = alter_spec(altered_wave2)  # resample the spectrum
    diff_spec = altered_spec2 - scaled_spec1  # difference spectrum
    return diff_spec.std()  # this is what we are minimizing

Now do the fit. Try a 2nd-degree polynomial to start; the coefficient array can be any size, depending on what degree polynomial you wish to fit.

In [ ]:
pixel_fit = minimize(fit_metric, np.array([0., 1., 0.]), args=(wave2[in_wave2],), method='Nelder-Mead')

The fit did indeed find a result very close to the expected [0, 1, 0].

In [ ]:
print(fit_metric(pixel_fit.x, wave2[in_wave2]))
print(pixel_fit)

Compare the two spectra, the difference between them, and the wavelengths

In [ ]:
# How did we do?
alt_wave2 = alter_wave(pixel_fit.x, wave2[in_wave2])
alt_spec2 = alter_spec(alt_wave2)

fig9 = plt.figure(figsize=(8, 4))
tmp = plt.plot(out_wave, scaled_spec1)
tmp = plt.plot(out_wave, alt_spec2, '--')

fig10 = plt.figure(figsize=(8, 4))
tmp = plt.plot(out_wave, interp_spec2 - scaled_spec1)
tmp = plt.plot(out_wave, alt_spec2 - scaled_spec1, '--')

fig11 = plt.figure(figsize=(8, 4))
tmp = plt.plot(wave2[in_wave2], alt_wave2, '.')
tmp = plt.plot(wave2[in_wave2], wave2[in_wave2], '--')

***
<a id='Trying_again'></a>
## Try again with a bad wavelength array

Now we'll try it with a bad wavelength array. We'll scale the wavelength array by 10% and try again.

In [ ]:
bad_wave2 = wave2[in_wave2] * 1.1

pixel_fit2 = minimize(fit_metric, np.array([0., 1.0, 0.]), args=(bad_wave2,), method='Nelder-Mead')
print(pixel_fit2)

The result found a scale factor almost exactly 1/1.1.

In [ ]:
alt_wave2a = alter_wave(pixel_fit2.x, bad_wave2)
alt_spec2a = alter_spec(alt_wave2a)

fig9 = plt.figure(figsize=(8, 4))
tmp = plt.plot(out_wave, scaled_spec1)
tmp = plt.plot(alt_wave2a, spec2[in_wave2], '--')

fig10 = plt.figure(figsize=(8, 4))
tmp = plt.plot(out_wave, interp_spec2 - scaled_spec1)
tmp = plt.plot(out_wave, alt_spec2a - scaled_spec1, '--')

fig11 = plt.figure(figsize=(8, 4))
tmp = plt.plot(wave2[in_wave2], alt_wave2a)
tmp = plt.plot(wave2[in_wave2], wave2[in_wave2], '--')

***
<a id='Versions'></a>
## Version information

```Versions:
  python:  sys.version_info(major=3, minor=6, micro=6, releaselevel='final', serial=0)
  astropy:  3.2.dev23244
  scipy:  1.1.0
  numpy:  1.15.4
```

In [ ]:
import sys, astropy, numpy, scipy
print('Versions:')
print('  python: ', sys.version_info)
print('  astropy: ', astropy.__version__)
print('  scipy: ', scipy.__version__)
print('  numpy: ', np.__version__)

***
<a id='about'></a>
## About this notebook

This notebook was created by Graham Kanarek, and revised by Henry Ferguson (STScI). Please send queries to the [JWST Help Desk](https://stsci.service-now.com/jwst).

Updated on 13 December 2018